In [ ]:
# 국민신문고 크롤링
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
browser_s = webdriver.Chrome('c:/matlab/data/chromedriver.exe')
url = "https://www.epeople.go.kr/jsp/user/pp/UPpProposOpenList.paid?flag=A"
browser_s.get(url)

def main_contents(C_no) :
    T_id = '''//*[@id="ancDetail_'''+C_no+'''"]'''
    T_ori = browser_s.find_element_by_xpath(T_id).get_attribute("onclick")[20:35]
    #글 고유번호 추출
    T_url = 'http://www.epeople.go.kr/jsp/user/pp/UPpProposOpenRead.paid?petiNo='+T_ori
    new_url = '''window.open("''' + T_url + '''","_blank");'''
    browser_s.execute_script(new_url)
    browser_s.switch_to.window(browser_s.window_handles[1])
    time.sleep(4)
    T_contents = browser_s.find_elements_by_class_name("view")[0].text
    T_contents = T_contents.replace('\n','').replace('현황 및 문제점','현황 및 문제점|').replace('개선방안','|개선방안|').replace('기대효과','|기대효과|')
    return T_contents

f = open("c:/python/test_sinmungo.txt", 'a',encoding='utf-8') 
browser_s.switch_to.window(browser_s.window_handles[0])
start_time = time.time()
for i in range(1,3):
    time.sleep(3)
    list_all = browser_s.find_elements_by_xpath("//*[@id='Form']/div[2]/table/tbody/tr/td")
    #현재 페이지의 테이블 요소(td) 선택
    for j in range(0,len(list_all)) :
        f.write(list_all[j].text+'|')
        if (j+1)%6 == 0: 
            f.write(main_contents(list_all[j-5].text))
            f.write('\n')
            browser_s.close()
            browser_s.switch_to.window(browser_s.window_handles[0])

        else : pass
    #테이블을 번호,제목,처리기관명,신청일,추진현황,조회 + 본문으로 저장
    btn_next = browser_s.find_element_by_xpath("//a[@class='btn next']").click()
    #다음 페이지 버튼 클릭
f.close()

In [ ]:
# coding: utf-8
import sys
sys.path.append('..')
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.font_manager as fm
%matplotlib inline
font_path = 'C:/Windows/Fonts/malgun.ttf'
font_name = fm.FontProperties(fname=font_path, size=10).get_name()
plt.rc('font', family=font_name, size=12)
from dataset import sequence
from common.optimizer import Adam
from common.trainer import Trainer
from common.util import eval_seq2seq
from seq2seq import Seq2seq
from peeky_seq2seq import PeekySeq2seq


# 데이터셋 읽기
(x_train, t_train), (x_test, t_test) = sequence.load_data('test_sinmungo.txt')
char_to_id, id_to_char = sequence.get_vocab()

# 입력 반전 여부 설정 =============================================
is_reverse = True
if is_reverse:
    x_train, x_test = x_train[:, ::-1], x_test[:, ::-1]
# ================================================================

# 하이퍼파라미터 설정
vocab_size = len(char_to_id)
wordvec_size = 16
hideen_size = 128
batch_size = 128
max_epoch = 25
max_grad = 5.0

# 일반 혹은 엿보기(Peeky) 설정 =====================================
# model = Seq2seq(vocab_size, wordvec_size, hideen_size)
model = PeekySeq2seq(vocab_size, wordvec_size, hideen_size)
# ================================================================
optimizer = Adam()
trainer = Trainer(model, optimizer)

peeky_acc_list = []
for epoch in range(max_epoch):
    trainer.fit(x_train, t_train, max_epoch=1,
                batch_size=batch_size, max_grad=max_grad, eval_interval=150)

    correct_num = 0
    for i in range(len(x_test)):
        question, correct = x_test[[i]], t_test[[i]]
        verbose = i < 10
        correct_num += eval_seq2seq(model, question, correct,
                                    id_to_char, verbose, is_reverse)

    acc = float(correct_num) / len(x_test)
    peeky_acc_list.append(acc)
    print('검증 정확도 %.3f%%' % (acc * 100))

In [ ]:
# 그래프 그리기
x_peeky = np.arange(len(peeky_acc_list))
plt.plot(x_reverse, peeky_acc_list, marker='o')
plt.xlabel('에폭')
plt.ylabel('정확도')
plt.ylim(0, 1.0)
plt.show()